<a href="https://colab.research.google.com/github/so-dipe/Data-Science-and-Machine-Learning-Projects/blob/main/UltraMNIST%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/   # If this returns an error, add or remove a space between .json and ~/.kaggle NB: there must always be space between them
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c ultra-mnist

In [ ]:
! unzip ultra-mnist.zip

In [ ]:
! pip install category_encoders

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense, Reshape, TimeDistributed, LSTM, Reshape
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from glob import glob
import cv2
from category_encoders import OneHotEncoder
#import tensorflow_datasets as tfds

In [ ]:
(X_train, y_train), (X_val, y_val) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
y_train_series = pd.Series(y_train)
y_train_cat = pd.get_dummies(y_train)
y_val_cat = pd.get_dummies(pd.Series(y_val))

In [ ]:
image_df = pd.read_csv('train.csv')
train_paths = list(glob('/content/train/*.jpeg'))

In [ ]:
def create_imageseq(imgpath):
    croppedimgs = list()
    image = cv2.imread(imgpath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edge = cv2.Canny(blur, 0, 0)
    contours, hierarachy = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    im2 = image.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # Drawing a rectangle on copied image
        rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Cropping the text block for giving input to OCR
        cropped = image[y:y + h, x:x + w]
        cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
        if cropped.shape[0] > 20 and cropped.shape[1] > 20:
            croppedimgs.append(cropped)

    return croppedimgs, im2

In [ ]:
def scale_resize_image(image):
    image = tf.image.convert_image_dtype(image, tf.float32) # equivalent to dividing image pixels by 255
    image = image[tf.newaxis, ..., tf.newaxis][0]
    image = tf.image.resize_with_pad(image, 100, 100)
    return image

In [ ]:
def createXY(imgarray):
    X = list()
    y = list()
    for imgdict in imgarray:
        X.append(imgdict['images'])
        y.append(imgdict['class'])
    y = pd.DataFrame(y).astype(str)
    ohe = OneHotEncoder(use_cat_names=True).fit(y)
    y_en = ohe.transform(y)
    return X, y, y_en

In [ ]:
def maxi(X):
  max = 0
  for i in range(len(X)):
    if max < len(X[i]):
      max = len(X[i])
  return max

In [ ]:
def loadseq(imagepath, df, num=100, encode=True):
    images = list()
    imagespath = glob(imagepath + "/*.jpeg")
    for file in imagespath[:num]:
      image_seq = dict()
      imagearray, im2 = create_imageseq(file)
      cropped_imgarray = [scale_resize_image(image) for image in imagearray]
      image_seq['images'] = cropped_imgarray
      image_seq['class'] = df[df['id'] == file[len(imagepath)+1:-5]]['digit_sum'].values[0]
      # image_seq['class'] = df[df['id'].str.contains(file)]['digit_sum'].values[0]
      images.append(image_seq)
    X, y, y_en = createXY(images)
    max = maxi(X)
    for im in X:
      if len(im) < max:
        fill = max - len(im)
        for i in range(fill):
          im.append(tf.constant(np.zeros((100, 100, 1))))
    X = tf.constant(np.asanyarray(X))
    if encode == True:
      return X, y, y_en
    else:
      return X, y

In [ ]:
X, y, y_en = loadseq('/content/train', image_df, 10)

In [ ]:
image_shape = (100, 100, 1)
kernel_size = (3, 3)
pool_size = (2, 2)
filters = 32
img_input = tf.keras.layers.Input(shape=image_shape)
img_conc = tf.keras.layers.Concatenate()([img_input, img_input, img_input])    
convNet = Sequential([
    tf.keras.applications.MobileNetV2(input_shape=img_conc,
                                               include_top=False,
                                               weights='imagenet'),
    Dense(11),
    Activation('softmax')
])

convNet.summary()
convNet.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

TypeError: ignored

In [ ]:
X_train_rescale = list()
for i in range(len(X_train)):
  X_train_rescale.append(scale_resize_image(X_train[i]))  
X_train_rescale = np.asanyarray(X_train_rescale)

In [ ]:
X_val_rescale = list()
for i in range(len(X_val)):
  X_val_rescale.append(scale_resize_image(X_val[i]))  
X_val_rescale = np.asanyarray(X_val_rescale)

In [ ]:
y_val_cat['none'] = 0
y_train_cat['none'] = 0

In [ ]:
convNet.fit(
    X_train_rescale, 
    y_train_cat, 
    epochs=10, 
    validation_data=(X_val_rescale, y_val_cat)
)

Epoch 1/10
1612/1875 [========================>.....] - ETA: 3:25 - loss: 0.2524 - accuracy: 0.9257

KeyboardInterrupt: ignored

In [ ]:
classification_layers = [
    TimeDistributed(convNet, input_shape=(58, 50, 50, 1)),
    Reshape((11, 58)),
    LSTM(4, activation='relu', input_shape=(11,)),
    Dense(128, activation='relu'),
    Dense(28, activation='softmax')
]
model = Sequential(classification_layers)
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])